In [1]:
import numpy as np
import pandas as pd
from gensim.models import word2vec

日本語学習済みのword2vecモデルを使う：
- http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/
- https://github.com/shiroyagicorp/japanese-word2vec-model-builder

きごさい歳時記から季語をとってきて、この内で学習済みモデルに含まれている季語をベクトル化する
- http://kigosai.sub.jp/27701-2
- `kigo.csv`にはベクトルと季語が入っている
- `n`はその季語が登場する俳句の数

- キーワードのリストを受け取って最もマッチする季語を返す関数
    - `words`がキーワードのリスト
    - `top_n`は季語の出現頻度上位何位までを利用するか
    - `top_n`を大きくしたほうが写真により近い季語が選出されるが、その季語を用いた訓練データの俳句の数が少ないものが選出され、生成される俳句の質が悪くなる可能性がある

In [41]:
def to_kigo(words, top_n=50):
    model = word2vec.Word2Vec.load('word2vec.gensim.model')
    df = pd.read_csv('kigo.csv')
    df = df.iloc[:top_n, :]
    V = df.drop(['kigo', 'n'], axis=1).values # 季語のベクトルを取り出す
    kigo = df.kigo.values # 季語のラベルを取り出す
    
    idx = 0
    max_similarity = 0
    for word in words:
        similarities = model.wv.similarity(kigo, word)
        if similarities.max() >= max_similarity:
            idx = similarities.argmax()
    
    return(kigo[idx])

例として以下を用いる

In [23]:
w1 = ['水', '森']
#w2 = ['空', '時計']
#w3 = ['水面', 'ビーチ']

In [42]:
to_kigo(w1, top_n=100)

'蛍'